In [27]:
# %pip install wandb
# %pip install datasets
# %pip install tokenizers
# %pip install -U transformers datasets tokenizers accelerate

In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import wandb
import math
import time

In [ ]:
from torch.nn import RMSNorm
from torch.utils.data import DataLoader
from datasets import load_dataset
from dataclasses import dataclass
from tokenizers import Tokenizer
from transformers import AutoTokenizer
from pathlib import Path
from tqdm import tqdm

c:\Prefix\solvingpapers\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'transformers'

In [30]:
@dataclass
class modelargs:
    block_size = 512
    batch_size = 32
    embeddings_dim = 512
    attn_dropout = 0.1
    vocab_size = 50257 #gpt2 bpe
    ignore_pad_token_in_loss = False #using packed blocks so padding never happens
    heads = 8 
    dropout = 0.1
    epochs = 1
    max_lr = 6e-4
    decoder_layers = 6
    weight_decay = 0.1
    beta1 = 0.9
    beta2 = 0.95
    clip=1.0
    experts = 8
    top_experts = 2
    use_shared_experts = True
    base_freq = 100000
    noisy_topk = False
    eps: float = 1e-8
    loss_scale = 0.03 
    use_aux_free_load_balancing = True
    aux_free_bias_update_rate = 0.001
    mtp_heads = 1
    latent_dim = 64

In [31]:
dataset = load_dataset('roneneldan/TinyStories')

In [32]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('gpt2', use_fast=True)
tokenizer.pad_token = tokenizer.eos_token #for batching

In [33]:
# vocab
vocab_size = tokenizer.vocab_size
vocab_size

50257

In [34]:
def encode(text: str):
    return tokenizer.encode(text, add_special_tokens = False)

In [ ]:
artifacts = Path('artifacts')
artifacts.mkdir(exist_ok=True)

tokenized_dir = artifacts/'tokenized'
tokenized_dir.mkdir(exist_ok=True)

In [ ]:
def chunked_tokenize(texts, chunk_size=10000):
    all_ids = []
    for i in tqdm(range(0, len(texts), chunk_size), desc='Tokenizing'):
        chunk = texts[i:i + chunk_size]
        chunk_text = '\n'.join(chunk)
        chunk_ids = encode(chunk_text)
        all_ids.extend(chunk_ids)
    return torch.tensor(all_ids, dtype=torch.long)

In [ ]:
train_path = tokenized_dir / 'train_ids.pt'
val_path = tokenized_dir / 'val_ids.pt'

train_ids = chunked_tokenize(dataset['train']['text'])
val_ids = chunked_tokenize(dataset['validation']['text'])

torch.save(train_ids, train_path)
torch.save(val_ids, val_path)